<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/380_GCO_ViolationDetection_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""Violation detection utilities for Governance & Compliance Orchestrator

Detects policy violations and generates compliance events.
"""

from typing import Dict, Any, List
from datetime import datetime


def generate_compliance_event(
    evaluation: Dict[str, Any],
    event_counter: int
) -> Dict[str, Any]:
    """
    Generate a compliance event from a policy evaluation that indicates a violation.

    Args:
        evaluation: Policy evaluation result with violation=True
        event_counter: Counter for generating unique compliance event IDs

    Returns:
        Compliance event dict
    """
    compliance_event_id = f"cmp_{event_counter:04d}"

    # Determine recommended action based on severity
    severity = evaluation.get("severity", "medium")
    required_action = evaluation.get("required_action", "review")

    recommended_actions = {
        "critical": "Immediate escalation to compliance officer and block action",
        "high": "Escalate to compliance officer",
        "medium": "Require human review before next action",
        "low": "Log for review and request explanation"
    }

    recommended_action = recommended_actions.get(severity, "Review and document")

    return {
        "compliance_event_id": compliance_event_id,
        "event_id": evaluation.get("event_id"),
        "risk_type": "policy_violation",
        "policy_id": evaluation.get("policy_id"),
        "severity": severity,
        "status": "open",
        "recommended_action": recommended_action,
        "reason": evaluation.get("reason", "Policy violation detected"),
        "timestamp": datetime.utcnow().isoformat() + "Z"
    }


def detect_violations(
    policy_evaluations: List[Dict[str, Any]]
) -> List[Dict[str, Any]]:
    """
    Detect all violations from policy evaluations and generate compliance events.

    Args:
        policy_evaluations: List of policy evaluation results

    Returns:
        List of compliance events (one per violation)
    """
    violations = [e for e in policy_evaluations if e.get("violation", False)]

    compliance_events = []
    event_counter = 1

    for violation in violations:
        compliance_event = generate_compliance_event(violation, event_counter)
        compliance_events.append(compliance_event)
        event_counter += 1

    return compliance_events

